# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [272]:
# import libraries
import pandas as pd
from sqlalchemy import create_engine
import re
import random
from sklearn.pipeline import Pipeline
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputClassifier
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.ensemble import RandomForestClassifier
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.metrics import f1_score, accuracy_score, recall_score
from sklearn.model_selection import GridSearchCV

In [273]:
# load data from database
# connect to the database
conn = create_engine('sqlite:///ETL_dataset.db')
df = pd.read_sql("SELECT * FROM ETL_dataset", conn)
X = df.message.values
Y = df.iloc[:,4:]

### 2. Write a tokenization function to process your text data

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [274]:
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
def tokenize(text):
    detected_urls = re.findall(url_regex, text)
    for url in detected_urls:
        text = text.replace(url, "urlplaceholder")

    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()

    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens
pipeline = Pipeline(steps = [
#    ('processing', TextProcessor()),
#    ('doc2vec':'pipeline_d2v'),
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(
            estimator=RandomForestClassifier(n_estimators=10))
        )
],
    )

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [275]:
X_train, X_test, y_train, y_test = train_test_split(X, Y)
pipeline.fit(X_train, y_train)

KeyboardInterrupt: 

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [135]:
pred = pipeline.predict(X_test)
for i in range(0,Y.shape[1]):
    print(y_test.keys()[i])
    print(classification_report(y_test.iloc[:,i], pred[:,i]))
    

related
              precision    recall  f1-score   support

           0       0.62      0.34      0.44      1531
           1       0.82      0.94      0.87      4977
           2       0.50      0.09      0.15        46

   micro avg       0.79      0.79      0.79      6554
   macro avg       0.65      0.45      0.49      6554
weighted avg       0.77      0.79      0.77      6554

request
              precision    recall  f1-score   support

           0       0.89      0.98      0.93      5443
           1       0.83      0.39      0.53      1111

   micro avg       0.88      0.88      0.88      6554
   macro avg       0.86      0.69      0.73      6554
weighted avg       0.88      0.88      0.86      6554

offer
              precision    recall  f1-score   support

           0       0.99      1.00      1.00      6521
           1       0.00      0.00      0.00        33

   micro avg       0.99      0.99      0.99      6554
   macro avg       0.50      0.50      0.50      655

/home/sirinebouslama/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [276]:
X = df.iloc[:,0:2]
Y = df.iloc[:,4:]
class TextProcessor( BaseEstimator, TransformerMixin ):
    """
    Class for carrying all the text pre-processing stuff throughout the project
    """

    def __init__(self):
        self.stopwords = stopwords.words('english')
        self.ps = PorterStemmer()

        # stemmer will be used for each unique word once
        self.stemmed = dict()

    def process(self, text: str, allow_stopwords: bool = False) -> str:
        """
        Process the specified text,
        splitting by non-alphabetic symbols, casting to lower case,
        removing stopwords, HTML tags and stemming each word

        :param text: text to process
        :param allow_stopwords: whether to remove stopwords
        :return: processed text
        """
        ret = []
        url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
        detected_urls = re.findall(url_regex, text)
        for url in detected_urls:
            text = text.replace(url, "urlplaceholder")
        # split and cast to lower case
        text = re.sub(r'<[^>]+>', ' ', str(text))
        for word in re.split('[^a-zA-Z]', str(text).lower()):
            # remove non-alphabetic and stop words
            if (word.isalpha() and word not in self.stopwords) or allow_stopwords:
                if word not in self.stemmed:
                    self.stemmed[word] = self.ps.stem(word)
                # use stemmed version of word
                ret.append(self.stemmed[word])
        return ' '.join(ret)
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        X['message'] = X['message'].apply(lambda x: self.process(x, allow_stopwords=True))
        return X

In [296]:
class doc2vec_transform(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass
    
    def train_d2v(self, df: pd.DataFrame, dim: int) -> (Doc2Vec, dict):
        """
        Train Doc2Vec object on provided data
        :param df: data to work with
        :param target: column name of target entity in df to train embeddings for
        :param features: list of feature names to be used for training
        :param dim: dimension of embedding vectors to train
        :return: trained Doc2Vec object
        """
        prepared = [TaggedDocument(s.split(), [str(i)]) for i, s in enumerate(df['message'])]
        d2v = Doc2Vec(prepared, vector_size=dim, workers=4, epochs=10, dm=0)
        docvecs = {d2v.docvecs.index2entity[i]: d2v.docvecs.vectors_docs[i]
                   for i in range(len(d2v.docvecs.index2entity))}
        return d2v, docvecs


    def pipeline_d2v(self, df: pd.DataFrame, dim: int):
        """
        Pipeline for training embeddings for messages via doc2vec algorithm

        :param df: raw df.csv dataset
        :param dim: dimension of doc2vec embeddings to train
        :return: trained msgs and Doc2Vec model
        """

        # train and save message embeddings
        _, msg_embs = self.train_d2v(df, dim)
        msg_embs = pd.DataFrame(msg_embs).transpose()
        return msg_embs
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        
        emb = self.pipeline_d2v(X, dim=10)
        return emb

In [297]:
clf = RandomForestClassifier(n_estimators=10)
clf.get_params().keys()

dict_keys(['bootstrap', 'class_weight', 'criterion', 'max_depth', 'max_features', 'max_leaf_nodes', 'min_impurity_decrease', 'min_impurity_split', 'min_samples_leaf', 'min_samples_split', 'min_weight_fraction_leaf', 'n_estimators', 'n_jobs', 'oob_score', 'random_state', 'verbose', 'warm_start'])

In [281]:
class Debug(BaseEstimator, TransformerMixin):

    def transform(self, X):
        print(X.shape)
        # what other output you want
        return X

    def fit(self, X, y=None, **fit_params):
        return self

In [285]:
pipeline2 = Pipeline(steps = [
   ('processing', TextProcessor()),
   ('doc2vec',doc2vec_transform()),
   ('debug2',Debug()),

   ('clf', MultiOutputClassifier(
            estimator=RandomForestClassifier(n_estimators=10)
            )
    )
],
)

In [286]:
X_train, X_test, y_train, y_test = train_test_split(X, Y)
pipeline2.fit(X_train, y_train)

/home/sirinebouslama/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



(0, 0)


ValueError: Found array with 0 sample(s) (shape=(0, 0)) while a minimum of 1 is required.

In [257]:
pred = pipeline2.predict(X_test)
for i in range(0,Y.shape[1]):
    print(y_test.keys()[i])
    print(classification_report(y_test.iloc[:,i], pred[:,i]))
    

/home/sirinebouslama/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


(6554, 10)
related
              precision    recall  f1-score   support

           0       0.18      0.11      0.14      1566
           1       0.74      0.84      0.79      4937
           2       0.56      0.10      0.17        51

   micro avg       0.66      0.66      0.66      6554
   macro avg       0.49      0.35      0.36      6554
weighted avg       0.61      0.66      0.63      6554

request
              precision    recall  f1-score   support

           0       0.84      0.99      0.91      5470
           1       0.26      0.02      0.04      1084

   micro avg       0.83      0.83      0.83      6554
   macro avg       0.55      0.50      0.47      6554
weighted avg       0.74      0.83      0.76      6554

offer
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6530
           1       0.00      0.00      0.00        24

   micro avg       1.00      1.00      1.00      6554
   macro avg       0.50      0.50      0.

/home/sirinebouslama/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [265]:
parameters = {  
        'clf__estimator__min_samples_split': [2, 4],
        'clf__estimator__max_features': ['log2', 'auto', 'sqrt', None],
        'clf__estimator__criterion': ['gini', 'entropy'],
        'clf__estimator__max_depth': [None, 25, 50, 100, 150, 200],
    }
cv = GridSearchCV(estimator=pipeline2, param_grid=parameters)
cv.fit(X_train,y_train)

/home/sirinebouslama/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:2052: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
/home/sirinebouslama/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


(13108, 10)


/home/sirinebouslama/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


(6554, 10)


/home/sirinebouslama/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


...

/home/sirinebouslama/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


(13108, 10)


/home/sirinebouslama/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


(13108, 10)


KeyboardInterrupt: 

In [262]:
pred = cv.predict(X_test)
for i in range(0,Y.shape[1]):
    print(y_test.keys()[i])
    print(classification_report(y_test.iloc[:,i], pred[:,i]))

/home/sirinebouslama/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


(6554, 10)
related
              precision    recall  f1-score   support

           0       0.13      0.05      0.07      1566
           1       0.74      0.90      0.82      4937
           2       0.18      0.04      0.06        51

   micro avg       0.69      0.69      0.69      6554
   macro avg       0.35      0.33      0.32      6554
weighted avg       0.59      0.69      0.63      6554

request
              precision    recall  f1-score   support

           0       0.84      0.72      0.77      5470
           1       0.17      0.29      0.21      1084

   micro avg       0.65      0.65      0.65      6554
   macro avg       0.50      0.50      0.49      6554
weighted avg       0.73      0.65      0.68      6554

offer
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6530
           1       0.00      0.00      0.00        24

   micro avg       1.00      1.00      1.00      6554
   macro avg       0.50      0.50      0.

/home/sirinebouslama/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [ ]:
cv.best_params_

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.